# Module 07: Meta-Analysis Basics

**Estimated Time**: 50 minutes

## Learning Objectives

By the end of this module, you will be able to:

1. **Understand** when meta-analysis is appropriate vs. narrative synthesis
2. **Calculate** common effect size metrics (Cohen's d, odds ratios, correlations)
3. **Implement** fixed-effect and random-effects meta-analysis models
4. **Assess** heterogeneity using I² and Q statistics
5. **Create** and interpret forest plots
6. **Detect** publication bias using funnel plots and statistical tests
7. **Conduct** subgroup analyses and meta-regression
8. **Report** meta-analysis results following PRISMA standards

## Why This Matters

**Meta-analysis is the quantitative synthesis of research findings.**

Advantages over narrative reviews:
- **Objective**: Uses statistical methods, not subjective interpretation
- **Precise**: Combines data for narrower confidence intervals
- **Powerful**: Detects effects missed by individual studies
- **Comprehensive**: Summarizes entire body of evidence

Meta-analyses:
- Guide clinical practice and policy decisions
- Identify research gaps
- Resolve contradictory findings
- Sit at the **top of the evidence hierarchy**

This module teaches you to conduct and interpret basic meta-analyses.

In [ ]:
# Import required libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.stats import norm, chi2
import warnings

warnings.filterwarnings("ignore")

# Set style
plt.style.use("seaborn-v0_8-darkgrid")
sns.set_palette("husl")

# Set random seed
np.random.seed(42)

# Create output directory
import os

os.makedirs("outputs/module_07", exist_ok=True)

print("✓ Libraries imported successfully")
print("✓ Output directory created")

## 1. When to Conduct Meta-Analysis

### Criteria for Meta-Analysis

Meta-analysis is appropriate when studies are:

1. **Sufficiently similar** (clinically and methodologically)
   - Same population (or similar enough)
   - Same intervention/exposure
   - Same outcome measure (or comparable)
   - Similar study designs

2. **Quantitatively comparable**
   - Report sufficient statistical information
   - Use compatible outcome measures
   - Comparable timepoints

### Meta-Analysis vs. Narrative Synthesis

| Factor | Meta-Analysis | Narrative Synthesis |
|--------|---------------|---------------------|
| **Similarity of studies** | High | Low to moderate |
| **Outcome measures** | Same or convertible | Heterogeneous |
| **Statistical data** | Available | Incomplete |
| **Study designs** | Similar | Mixed |
| **Number of studies** | ≥2 (preferably ≥5) | Any |
| **Clinical heterogeneity** | Low to moderate | High |

### When NOT to Meta-Analyze

❌ Studies too heterogeneous ("apples and oranges")  
❌ Insufficient statistical information  
❌ Different outcome measures that can't be standardized  
❌ Only 1-2 studies (too few to be meaningful)  
❌ Mix of very high and very low quality studies  

**Remember**: "Garbage in, garbage out" — poor studies produce poor meta-analyses.

## 2. Effect Size Calculation

**Effect size** = Standardized measure of the magnitude of an effect

### Common Effect Sizes

#### 1. Cohen's d (Standardized Mean Difference)

**When to use**: Comparing continuous outcomes between two groups

$$d = \frac{\bar{X}_1 - \bar{X}_2}{SD_{\text{pooled}}}$$

Where:
$$SD_{\text{pooled}} = \sqrt{\frac{(n_1 - 1)SD_1^2 + (n_2 - 1)SD_2^2}{n_1 + n_2 - 2}}$$

**Interpretation**:
- |d| = 0.2: Small effect
- |d| = 0.5: Medium effect
- |d| = 0.8: Large effect

#### 2. Odds Ratio (OR)

**When to use**: Dichotomous outcomes (success/failure, yes/no)

$$OR = \frac{\text{Odds}_{\text{treatment}}}{\text{Odds}_{\text{control}}} = \frac{(a/b)}{(c/d)}$$

From 2×2 table:
```
           Event    No Event
Treatment    a         b
Control      c         d
```

**Interpretation**:
- OR = 1: No effect
- OR > 1: Increased odds in treatment group
- OR < 1: Decreased odds in treatment group

#### 3. Correlation Coefficient (r)

**When to use**: Association between two continuous variables

**Range**: -1 to +1

**Interpretation**:
- |r| = 0.1: Small
- |r| = 0.3: Medium
- |r| = 0.5: Large

### Standard Error (SE) of Effect Sizes

Needed for weighting studies in meta-analysis.

**Cohen's d**:
$$SE_d = \sqrt{\frac{n_1 + n_2}{n_1 \cdot n_2} + \frac{d^2}{2(n_1 + n_2)}}$$

**Log OR**:
$$SE_{\log OR} = \sqrt{\frac{1}{a} + \frac{1}{b} + \frac{1}{c} + \frac{1}{d}}$$

**Fisher's z** (for correlation r):
$$z = 0.5 \ln\left(\frac{1+r}{1-r}\right), \quad SE_z = \frac{1}{\sqrt{n-3}}$$

In [ ]:
# Effect size calculation functions


def cohens_d(mean1, sd1, n1, mean2, sd2, n2):
    """
    Calculate Cohen's d and its standard error.

    Parameters:
    - mean1, sd1, n1: Mean, SD, and N for group 1
    - mean2, sd2, n2: Mean, SD, and N for group 2

    Returns:
    - d: Cohen's d
    - se_d: Standard error of d
    """
    # Pooled SD
    pooled_sd = np.sqrt(((n1 - 1) * sd1**2 + (n2 - 1) * sd2**2) / (n1 + n2 - 2))

    # Cohen's d
    d = (mean1 - mean2) / pooled_sd

    # Standard error
    se_d = np.sqrt((n1 + n2) / (n1 * n2) + d**2 / (2 * (n1 + n2)))

    return d, se_d


def odds_ratio(a, b, c, d):
    """
    Calculate odds ratio and its standard error.

    Parameters:
    - a, b, c, d: Cells of 2x2 table
                  Treatment: Event (a), No Event (b)
                  Control: Event (c), No Event (d)

    Returns:
    - or_value: Odds ratio
    - log_or: Log odds ratio
    - se_log_or: Standard error of log OR
    """
    # Odds ratio
    or_value = (a * d) / (b * c)

    # Log OR
    log_or = np.log(or_value)

    # Standard error of log OR
    se_log_or = np.sqrt(1 / a + 1 / b + 1 / c + 1 / d)

    return or_value, log_or, se_log_or


def fishers_z(r, n):
    """
    Transform correlation to Fisher's z and calculate SE.

    Parameters:
    - r: Correlation coefficient
    - n: Sample size

    Returns:
    - z: Fisher's z
    - se_z: Standard error of z
    """
    # Fisher's z transformation
    z = 0.5 * np.log((1 + r) / (1 - r))

    # Standard error
    se_z = 1 / np.sqrt(n - 3)

    return z, se_z


# Example calculations
print("EFFECT SIZE EXAMPLES")
print("=" * 80)

# Example 1: Cohen's d
print("\n1. Cohen's d (Depression scores: Exercise vs. Control)")
mean_exercise = 15.2
sd_exercise = 5.1
n_exercise = 50

mean_control = 20.8
sd_control = 6.3
n_control = 48

d, se_d = cohens_d(mean_exercise, sd_exercise, n_exercise, mean_control, sd_control, n_control)

print(f"   Exercise: M = {mean_exercise}, SD = {sd_exercise}, N = {n_exercise}")
print(f"   Control:  M = {mean_control}, SD = {sd_control}, N = {n_control}")
print(f"\n   Cohen's d = {d:.3f} (SE = {se_d:.3f})")
print(f"   95% CI: [{d - 1.96*se_d:.3f}, {d + 1.96*se_d:.3f}]")

if abs(d) >= 0.8:
    magnitude = "Large"
elif abs(d) >= 0.5:
    magnitude = "Medium"
elif abs(d) >= 0.2:
    magnitude = "Small"
else:
    magnitude = "Negligible"

print(f"   Interpretation: {magnitude} effect")

# Example 2: Odds Ratio
print("\n2. Odds Ratio (Recovery: Treatment vs. Control)")
print("\n   2×2 Table:")
print("                Recovered    Not Recovered")
print("   Treatment       35            15")
print("   Control         20            30")

a, b, c, d = 35, 15, 20, 30
or_val, log_or, se_log_or = odds_ratio(a, b, c, d)

print(f"\n   Odds Ratio = {or_val:.3f}")
print(f"   Log OR = {log_or:.3f} (SE = {se_log_or:.3f})")

# 95% CI for OR
ci_lower = np.exp(log_or - 1.96 * se_log_or)
ci_upper = np.exp(log_or + 1.96 * se_log_or)
print(f"   95% CI: [{ci_lower:.3f}, {ci_upper:.3f}]")
print(f"   Interpretation: Treatment has {or_val:.1f}x the odds of recovery vs. control")

# Example 3: Correlation
print("\n3. Fisher's z (Correlation between study time and grades)")
r = 0.45
n = 120
z, se_z = fishers_z(r, n)

print(f"   Correlation r = {r}")
print(f"   Sample size N = {n}")
print(f"   Fisher's z = {z:.3f} (SE = {se_z:.3f})")
print(f"   95% CI: [{z - 1.96*se_z:.3f}, {z + 1.96*se_z:.3f}]")

## 3. Fixed-Effect vs. Random-Effects Models

### Fixed-Effect Model

**Assumption**: All studies share a common true effect size.

**When to use**:
- Studies are methodologically identical
- Interest is limited to included studies only
- Low heterogeneity (I² < 25%)

**Pooled effect**:
$$\bar{\theta}_{\text{FE}} = \frac{\sum w_i \theta_i}{\sum w_i}$$

Where: $w_i = \frac{1}{SE_i^2}$ (inverse variance weights)

### Random-Effects Model

**Assumption**: True effect sizes vary across studies.

**When to use**:
- Studies differ in populations, methods, settings
- Want to generalize beyond included studies
- Moderate to high heterogeneity (I² ≥ 25%)

**Pooled effect** (DerSimonian-Laird method):
$$\bar{\theta}_{\text{RE}} = \frac{\sum w_i^* \theta_i}{\sum w_i^*}$$

Where: $w_i^* = \frac{1}{SE_i^2 + \tau^2}$ and $\tau^2$ = between-study variance

### Key Differences

| Aspect | Fixed-Effect | Random-Effects |
|--------|-------------|----------------|
| **True effect** | Single value | Distribution of values |
| **Weights** | Larger studies get more weight | More balanced weights |
| **CI width** | Narrower | Wider |
| **Generalization** | To similar studies only | To broader population |
| **Typical use** | Rare (restrictive assumption) | Common (more realistic) |

**Default recommendation**: Use random-effects unless you have strong reason to believe all studies share identical true effect.

In [ ]:
# Conduct fixed-effect and random-effects meta-analysis


def meta_analysis_fe(effects, ses):
    """
    Fixed-effect meta-analysis.

    Parameters:
    - effects: Array of effect sizes
    - ses: Array of standard errors

    Returns:
    - pooled_effect: Pooled effect size
    - pooled_se: Standard error of pooled effect
    - weights: Study weights
    """
    # Inverse variance weights
    weights = 1 / (ses**2)

    # Pooled effect
    pooled_effect = np.sum(weights * effects) / np.sum(weights)

    # SE of pooled effect
    pooled_se = np.sqrt(1 / np.sum(weights))

    return pooled_effect, pooled_se, weights


def meta_analysis_re(effects, ses):
    """
    Random-effects meta-analysis (DerSimonian-Laird).

    Parameters:
    - effects: Array of effect sizes
    - ses: Array of standard errors

    Returns:
    - pooled_effect: Pooled effect size
    - pooled_se: Standard error of pooled effect
    - weights: Study weights
    - tau_squared: Between-study variance
    """
    # Fixed-effect estimate (needed for tau calculation)
    w_fe = 1 / (ses**2)
    theta_fe = np.sum(w_fe * effects) / np.sum(w_fe)

    # Q statistic
    Q = np.sum(w_fe * (effects - theta_fe) ** 2)

    # Degrees of freedom
    k = len(effects)
    df = k - 1

    # Tau-squared (between-study variance)
    C = np.sum(w_fe) - np.sum(w_fe**2) / np.sum(w_fe)
    tau_squared = max(0, (Q - df) / C)  # Can't be negative

    # Random-effects weights
    weights = 1 / (ses**2 + tau_squared)

    # Pooled effect
    pooled_effect = np.sum(weights * effects) / np.sum(weights)

    # SE of pooled effect
    pooled_se = np.sqrt(1 / np.sum(weights))

    return pooled_effect, pooled_se, weights, tau_squared


# Simulate meta-analysis with 8 studies
np.random.seed(123)
n_studies = 8

# Study effect sizes (Cohen's d) with heterogeneity
true_effects = np.random.normal(-0.5, 0.2, n_studies)  # Mean d = -0.5, some variation
study_ns = np.random.randint(30, 150, n_studies)  # Sample sizes

# Observed effects with sampling error
observed_effects = true_effects + np.random.normal(0, 0.15, n_studies)

# Standard errors (depends on sample size)
study_ses = 0.3 / np.sqrt(study_ns)

# Create study data
study_data = pd.DataFrame(
    {
        "Study": [f"Study {i+1}" for i in range(n_studies)],
        "Author": [f"Author {chr(65+i)} et al." for i in range(n_studies)],
        "N": study_ns,
        "Effect_Size": observed_effects,
        "SE": study_ses,
    }
)

study_data["CI_Lower"] = study_data["Effect_Size"] - 1.96 * study_data["SE"]
study_data["CI_Upper"] = study_data["Effect_Size"] + 1.96 * study_data["SE"]

print("STUDY DATA FOR META-ANALYSIS")
print("=" * 80)
print(
    study_data[["Study", "Author", "N", "Effect_Size", "SE", "CI_Lower", "CI_Upper"]].to_string(
        index=False
    )
)

# Fixed-effect meta-analysis
fe_effect, fe_se, fe_weights = meta_analysis_fe(observed_effects, study_ses)

print("\n" + "=" * 80)
print("FIXED-EFFECT META-ANALYSIS")
print("=" * 80)
print(f"Pooled effect size: {fe_effect:.3f}")
print(f"Standard error: {fe_se:.3f}")
print(f"95% CI: [{fe_effect - 1.96*fe_se:.3f}, {fe_effect + 1.96*fe_se:.3f}]")
print(f"Z-test: z = {fe_effect/fe_se:.3f}, p = {2*(1-norm.cdf(abs(fe_effect/fe_se))):.4f}")

# Random-effects meta-analysis
re_effect, re_se, re_weights, tau_sq = meta_analysis_re(observed_effects, study_ses)

print("\n" + "=" * 80)
print("RANDOM-EFFECTS META-ANALYSIS")
print("=" * 80)
print(f"Pooled effect size: {re_effect:.3f}")
print(f"Standard error: {re_se:.3f}")
print(f"95% CI: [{re_effect - 1.96*re_se:.3f}, {re_effect + 1.96*re_se:.3f}]")
print(f"Z-test: z = {re_effect/re_se:.3f}, p = {2*(1-norm.cdf(abs(re_effect/re_se))):.4f}")
print(f"Between-study variance (τ²): {tau_sq:.4f}")

print("\n" + "=" * 80)
print("COMPARISON")
print("=" * 80)
print(
    f"Fixed-effect:   {fe_effect:.3f} [{fe_effect - 1.96*fe_se:.3f}, {fe_effect + 1.96*fe_se:.3f}]"
)
print(
    f"Random-effects: {re_effect:.3f} [{re_effect - 1.96*re_se:.3f}, {re_effect + 1.96*re_se:.3f}]"
)
print(f"\n💡 Random-effects CI is wider (accounts for between-study heterogeneity).")

## 4. Heterogeneity Assessment

**Heterogeneity** = Variability in effect sizes across studies beyond sampling error.

### Why It Matters
- High heterogeneity suggests studies aren't measuring the same thing
- May indicate need for subgroup analysis
- Informs model choice (fixed vs. random effects)

### I² Statistic

**Most commonly used heterogeneity measure**

$$I^2 = \frac{Q - df}{Q} \times 100\%$$

Where:
- Q = Cochran's Q statistic
- df = degrees of freedom (k - 1)

**Interpretation** (Higgins & Thompson, 2002):
- I² = 0-25%: Low heterogeneity
- I² = 25-50%: Moderate heterogeneity
- I² = 50-75%: Substantial heterogeneity
- I² > 75%: Considerable heterogeneity

### Cochran's Q Test

**Null hypothesis**: All studies share common effect size

$$Q = \sum w_i (\theta_i - \bar{\theta})^2$$

Follows chi-square distribution with k-1 degrees of freedom.

**Problem**: Low power when few studies; overly sensitive when many studies.

### Tau-squared (τ²)

**Between-study variance**

- τ² = 0: No heterogeneity
- τ² > 0: Presence of heterogeneity

**Note**: Unlike I², τ² depends on the effect size metric and scale.

In [ ]:
# Calculate heterogeneity statistics


def heterogeneity_stats(effects, ses):
    """
    Calculate heterogeneity statistics: Q, I², tau².

    Parameters:
    - effects: Array of effect sizes
    - ses: Array of standard errors

    Returns:
    - Dictionary with Q, df, p_value, I2, tau_squared
    """
    k = len(effects)
    df = k - 1

    # Fixed-effect weights
    w = 1 / (ses**2)

    # Fixed-effect pooled estimate
    theta_fe = np.sum(w * effects) / np.sum(w)

    # Q statistic
    Q = np.sum(w * (effects - theta_fe) ** 2)

    # P-value from chi-square distribution
    p_value = 1 - chi2.cdf(Q, df)

    # I² statistic
    I2 = max(0, (Q - df) / Q) * 100

    # Tau-squared
    C = np.sum(w) - np.sum(w**2) / np.sum(w)
    tau_squared = max(0, (Q - df) / C)

    return {"Q": Q, "df": df, "p_value": p_value, "I2": I2, "tau_squared": tau_squared}


# Calculate heterogeneity for our example data
het_stats = heterogeneity_stats(observed_effects, study_ses)

print("HETEROGENEITY ASSESSMENT")
print("=" * 80)
print(f"\nCochran's Q Test:")
print(f"  Q = {het_stats['Q']:.3f}")
print(f"  df = {het_stats['df']}")
print(f"  p-value = {het_stats['p_value']:.4f}")

if het_stats["p_value"] < 0.05:
    print(f"  ✓ Significant heterogeneity detected (p < .05)")
else:
    print(f"  ✗ No significant heterogeneity (p ≥ .05)")

print(f"\nI² Statistic:")
print(f"  I² = {het_stats['I2']:.1f}%")

if het_stats["I2"] < 25:
    interpretation = "Low heterogeneity"
elif het_stats["I2"] < 50:
    interpretation = "Moderate heterogeneity"
elif het_stats["I2"] < 75:
    interpretation = "Substantial heterogeneity"
else:
    interpretation = "Considerable heterogeneity"

print(f"  Interpretation: {interpretation}")

print(f"\nBetween-Study Variance (τ²):")
print(f"  τ² = {het_stats['tau_squared']:.4f}")
print(f"  τ = {np.sqrt(het_stats['tau_squared']):.4f} (SD of true effects)")

print("\n" + "=" * 80)
print("RECOMMENDATION")
print("=" * 80)

if het_stats["I2"] >= 50:
    print("⚠ Substantial heterogeneity present.")
    print("\nRecommendations:")
    print("  1. Use random-effects model")
    print("  2. Investigate sources via subgroup analysis or meta-regression")
    print("  3. Consider if meta-analysis is appropriate")
elif het_stats["I2"] >= 25:
    print("⚠ Moderate heterogeneity present.")
    print("\nRecommendations:")
    print("  1. Use random-effects model (more conservative)")
    print("  2. Explore potential moderators")
else:
    print("✓ Low heterogeneity.")
    print("\nRecommendations:")
    print("  1. Fixed-effect model may be appropriate")
    print("  2. Random-effects still preferred for generalizability")

## 5. Forest Plots

**Forest plot** = Standard visualization for meta-analysis results

### Components

1. **Study labels**: Author, year
2. **Effect sizes**: Numerical values
3. **Confidence intervals**: Horizontal lines
4. **Point estimates**: Squares (size ∝ weight)
5. **Pooled estimate**: Diamond at bottom
6. **Null line**: Vertical line at zero/one
7. **Scale**: X-axis showing effect size

### Interpretation

- **Square position**: Study's point estimate
- **Square size**: Study's weight/precision
- **Line width**: 95% confidence interval
- **Diamond**: Pooled effect and its CI
- **Crosses null line**: Non-significant result
- **All on one side**: Consistent direction

In [ ]:
# Create forest plot


def forest_plot(study_data, pooled_effect, pooled_se, model_name="Random-Effects"):
    """
    Create a forest plot for meta-analysis.

    Parameters:
    - study_data: DataFrame with columns: Study, Effect_Size, SE, CI_Lower, CI_Upper
    - pooled_effect: Pooled effect size
    - pooled_se: SE of pooled effect
    - model_name: Name of model (for title)
    """
    n_studies = len(study_data)

    fig, ax = plt.subplots(figsize=(12, 8))

    # Plot individual studies
    y_positions = np.arange(n_studies, 0, -1)

    for i, (idx, row) in enumerate(study_data.iterrows()):
        y = y_positions[i]

        # Confidence interval (horizontal line)
        ax.plot([row["CI_Lower"], row["CI_Upper"]], [y, y], "k-", linewidth=1.5, zorder=1)

        # Point estimate (square, size proportional to weight)
        weight = 1 / (row["SE"] ** 2)
        marker_size = 100 + weight * 50  # Scale for visibility
        ax.scatter(
            row["Effect_Size"],
            y,
            s=marker_size,
            marker="s",
            color="#2E86AB",
            edgecolors="black",
            linewidths=1.5,
            zorder=2,
            alpha=0.7,
        )

        # Study label
        ax.text(-1.2, y, row["Study"], ha="right", va="center", fontsize=10)

        # Effect size value
        ax.text(
            1.0,
            y,
            f"{row['Effect_Size']:.2f} [{row['CI_Lower']:.2f}, {row['CI_Upper']:.2f}]",
            ha="left",
            va="center",
            fontsize=9,
        )

    # Pooled estimate (diamond)
    pooled_ci_lower = pooled_effect - 1.96 * pooled_se
    pooled_ci_upper = pooled_effect + 1.96 * pooled_se

    diamond_y = -1
    diamond_height = 0.3

    # Diamond shape: left, top, right, bottom
    diamond_x = [pooled_ci_lower, pooled_effect, pooled_ci_upper, pooled_effect, pooled_ci_lower]
    diamond_y_coords = [
        diamond_y,
        diamond_y + diamond_height,
        diamond_y,
        diamond_y - diamond_height,
        diamond_y,
    ]

    ax.fill(
        diamond_x,
        diamond_y_coords,
        color="#E63946",
        edgecolor="black",
        linewidth=2,
        alpha=0.7,
        zorder=3,
    )

    # Pooled label
    ax.text(
        -1.2,
        diamond_y,
        f"{model_name} Model",
        ha="right",
        va="center",
        fontsize=11,
        fontweight="bold",
    )
    ax.text(
        1.0,
        diamond_y,
        f"{pooled_effect:.2f} [{pooled_ci_lower:.2f}, {pooled_ci_upper:.2f}]",
        ha="left",
        va="center",
        fontsize=10,
        fontweight="bold",
    )

    # Null effect line
    ax.axvline(x=0, color="gray", linestyle="--", linewidth=1.5, zorder=0)

    # Formatting
    ax.set_xlabel("Effect Size (Cohen's d)", fontsize=13, fontweight="bold")
    ax.set_ylabel("")
    ax.set_title(
        f"Forest Plot: Exercise for Depression\n({model_name} Meta-Analysis)",
        fontsize=14,
        fontweight="bold",
    )

    ax.set_ylim([-2, n_studies + 1])
    ax.set_xlim([-1.3, 1.5])
    ax.set_yticks([])
    ax.spines["left"].set_visible(False)
    ax.spines["right"].set_visible(False)
    ax.spines["top"].set_visible(False)
    ax.grid(True, alpha=0.3, axis="x")

    # Add "Favors Control" and "Favors Exercise" labels
    ax.text(-0.9, n_studies + 0.5, "Favors Control", ha="center", fontsize=10, style="italic")
    ax.text(0.9, n_studies + 0.5, "Favors Exercise", ha="center", fontsize=10, style="italic")

    plt.tight_layout()
    plt.savefig("outputs/module_07/forest_plot.png", dpi=300, bbox_inches="tight")
    plt.show()


# Create forest plot
forest_plot(study_data, re_effect, re_se, "Random-Effects")

print("\n✓ Forest plot saved to outputs/module_07/forest_plot.png")
print("\n💡 Forest plots are the standard way to visualize meta-analysis results.")
print("   They show both individual study effects and the overall pooled estimate.")

## 6. Publication Bias

**Publication bias** = Tendency for studies with positive/significant results to be published more often than negative/null findings.

### Consequences
- Overestimation of effect sizes
- False positives in meta-analyses
- Misleading clinical recommendations

### Detection Methods

#### 1. Funnel Plot

**Visual inspection**: Plot effect size vs. precision (1/SE)

**Expected pattern** (no bias): Symmetric funnel
- Small studies: Wide spread at bottom
- Large studies: Narrow spread at top

**Asymmetry suggests**:
- Small negative studies missing (publication bias)
- Heterogeneity
- True relationship between study size and effect

#### 2. Egger's Test

**Statistical test** for funnel plot asymmetry

Regresses standardized effect on precision:
$$\frac{\theta_i}{SE_i} = \beta_0 + \beta_1 \left(\frac{1}{SE_i}\right) + \varepsilon_i$$

**Null hypothesis**: $\beta_0 = 0$ (no asymmetry)

#### 3. Trim-and-Fill Method

**Estimate** number of missing studies and adjust pooled effect

### Mitigation Strategies

1. **Search grey literature**: Dissertations, conference papers, trial registries
2. **Contact authors**: Request unpublished data
3. **Protocol registration**: PROSPERO prevents post-hoc changes
4. **Sensitivity analysis**: Assess impact of potential bias

In [ ]:
# Create funnel plot and conduct Egger's test


def eggers_test(effects, ses):
    """
    Conduct Egger's test for publication bias.

    Parameters:
    - effects: Array of effect sizes
    - ses: Array of standard errors

    Returns:
    - bias: Intercept (bias estimate)
    - p_value: P-value for test
    """
    # Standardized effect
    y = effects / ses

    # Precision
    x = 1 / ses

    # Linear regression
    from scipy.stats import linregress

    slope, intercept, r_value, p_value, std_err = linregress(x, y)

    return intercept, p_value


# Conduct Egger's test
bias_intercept, egger_p = eggers_test(observed_effects, study_ses)

print("PUBLICATION BIAS ASSESSMENT")
print("=" * 80)
print("\nEgger's Test for Funnel Plot Asymmetry:")
print(f"  Bias (intercept): {bias_intercept:.3f}")
print(f"  P-value: {egger_p:.4f}")

if egger_p < 0.10:  # Liberal threshold
    print(f"  ⚠ Significant asymmetry detected (p < .10)")
    print(f"     Possible publication bias or other small-study effects.")
else:
    print(f"  ✓ No significant asymmetry (p ≥ .10)")

# Create funnel plot
fig, ax = plt.subplots(figsize=(10, 8))

# Plot studies
precision = 1 / study_ses
ax.scatter(
    observed_effects,
    precision,
    s=100,
    alpha=0.6,
    color="#2E86AB",
    edgecolors="black",
    linewidths=1.5,
)

# Pooled effect line
ax.axvline(
    x=re_effect,
    color="#E63946",
    linestyle="--",
    linewidth=2,
    label=f"Pooled Effect ({re_effect:.3f})",
)

# Funnel (expected distribution under no bias)
# Draw lines from pooled effect at various precisions
precision_range = np.array([min(precision), max(precision)])

for z in [1.96, 1.64]:  # 95% and 90% pseudo-CI
    # Upper bound
    upper = re_effect + z / precision_range
    # Lower bound
    lower = re_effect - z / precision_range

    ax.plot(
        [lower[0], re_effect],
        [precision_range[0], precision_range[1]],
        "gray",
        linestyle=":",
        linewidth=1,
        alpha=0.5,
    )
    ax.plot(
        [upper[0], re_effect],
        [precision_range[0], precision_range[1]],
        "gray",
        linestyle=":",
        linewidth=1,
        alpha=0.5,
    )

ax.set_xlabel("Effect Size (Cohen's d)", fontsize=13, fontweight="bold")
ax.set_ylabel("Precision (1/SE)", fontsize=13, fontweight="bold")
ax.set_title("Funnel Plot: Assessment of Publication Bias", fontsize=14, fontweight="bold")
ax.legend(loc="upper right", fontsize=11)
ax.grid(True, alpha=0.3)

# Add interpretation box
interpretation_text = (
    "Expected pattern (no bias):\n"
    "• Symmetric around pooled effect\n"
    "• Small studies spread widely\n"
    "• Large studies near pooled effect"
)
ax.text(
    0.02,
    0.98,
    interpretation_text,
    transform=ax.transAxes,
    fontsize=9,
    verticalalignment="top",
    bbox=dict(boxstyle="round", facecolor="wheat", alpha=0.3),
)

plt.tight_layout()
plt.savefig("outputs/module_07/funnel_plot.png", dpi=300, bbox_inches="tight")
plt.show()

print("\n✓ Funnel plot saved to outputs/module_07/funnel_plot.png")
print("\n💡 Funnel plot asymmetry can indicate publication bias,")
print("   but can also result from heterogeneity or chance.")

## 7. Practice Exercises

### Exercise 1: Calculate Effect Sizes

Given this data from an RCT:
- **Treatment group**: N=60, Mean=25.4, SD=5.2
- **Control group**: N=55, Mean=29.1, SD=6.1

Calculate:
1. Cohen's d
2. Standard error of d
3. 95% confidence interval

In [ ]:
# Exercise 2: Conduct Meta-Analysis
# Given 5 studies with the following effect sizes and SEs:

study_effects = np.array([0.35, 0.52, 0.41, 0.28, 0.47])
study_ses = np.array([0.15, 0.18, 0.12, 0.20, 0.14])

# YOUR TASK:
# 1. Conduct fixed-effect meta-analysis
# 2. Conduct random-effects meta-analysis
# 3. Calculate heterogeneity (I²)
# 4. Which model is more appropriate?

# YOUR CODE HERE

In [ ]:
# Exercise 3: Interpret Heterogeneity
# You conduct a meta-analysis and get I² = 68%

# QUESTIONS:
# 1. How would you interpret this level of heterogeneity?
# 2. What model should you use (fixed or random)?
# 3. What additional analyses should you consider?
# 4. Should you still pool the studies?

## 8. Summary and Key Takeaways

### The Meta-Analysis Process

```
1. EXTRACT EFFECT SIZES
   └─> Calculate d, OR, or r for each study
   └─> Compute standard errors

2. ASSESS HETEROGENEITY
   └─> Calculate I², Q, τ²
   └─> Decide: Fixed or random effects?

3. POOL EFFECT SIZES
   └─> Weight studies by precision
   └─> Calculate pooled estimate and CI

4. CREATE FOREST PLOT
   └─> Visualize individual and pooled effects

5. ASSESS PUBLICATION BIAS
   └─> Funnel plot
   └─> Egger's test

6. EXPLORE HETEROGENEITY
   └─> Subgroup analysis
   └─> Meta-regression

7. REPORT FINDINGS
   └─> Follow PRISMA guidelines
```

### Critical Decision Points

| Decision | Guideline |
|----------|----------|
| **Pool or not?** | Only if studies sufficiently similar |
| **Fixed vs. Random?** | Random effects (almost always safer) |
| **I² threshold?** | >50% → investigate heterogeneity |
| **Publication bias?** | Search grey literature, use caution |
| **Few studies (k<5)?** | Interpret with caution; CIs wide |

### Common Mistakes

✗ Pooling incompatible studies ("apples and oranges")  
✗ Ignoring heterogeneity  
✗ Using fixed-effect when random-effects appropriate  
✗ Not assessing publication bias  
✗ Over-interpreting small meta-analyses (k<5)  
✗ Mixing different outcome types  

### Best Practices

✓ Always calculate heterogeneity statistics  
✓ Default to random-effects model  
✓ Create forest plots for transparency  
✓ Assess publication bias  
✓ Report I², τ², and confidence intervals  
✓ Conduct sensitivity analyses  
✓ Investigate heterogeneity sources  

### Moving Forward

You now know how to conduct basic meta-analyses. The next module covers **Research Communication & Writing**, teaching you to present your findings effectively.

## 9. Additional Resources

### Essential Readings

1. **Borenstein et al. (2021)**. *Introduction to Meta-Analysis* (2nd ed.)
   - Comprehensive textbook

2. **Higgins & Green (2011)**. *Cochrane Handbook* (Chapter 9: Analysing data)
   - Gold standard methods

3. **Egger et al. (1997)**. "Bias in meta-analysis detected by funnel plot asymmetry"
   - Classic publication bias paper

### Software

- **RevMan** (Cochrane): Free, user-friendly
- **R packages**: meta, metafor, dmetar
- **Python**: metapy (limited)
- **Stata**: metan, metareg
- **Comprehensive Meta-Analysis (CMA)**: Commercial, powerful

### Online Calculators

- **Effect Size Calculator** (psychometrica.de)
- **Meta-Essentials** (Free Excel-based tool)

### Reporting

- **PRISMA Statement** (prisma-statement.org)
- **MOOSE Guidelines** (observational studies)

In [ ]:
# Save meta-analysis checklist

ma_checklist = pd.DataFrame(
    {
        "Phase": [
            "Preparation",
            "Preparation",
            "Preparation",
            "Analysis",
            "Analysis",
            "Analysis",
            "Analysis",
            "Assessment",
            "Assessment",
            "Assessment",
            "Reporting",
            "Reporting",
            "Reporting",
        ],
        "Task": [
            "Ensure studies are sufficiently similar",
            "Extract effect sizes and SEs from all studies",
            "Check data accuracy (double extraction)",
            "Calculate heterogeneity (I², Q, τ²)",
            "Choose model (fixed vs. random effects)",
            "Pool effect sizes",
            "Calculate pooled effect and 95% CI",
            "Create forest plot",
            "Create funnel plot",
            "Conduct Egger's test for publication bias",
            "Report all heterogeneity statistics",
            "Report pooled effect with interpretation",
            "Discuss limitations and heterogeneity sources",
        ],
        "Completed": [""] * 13,
    }
)

ma_checklist.to_csv("outputs/module_07/meta_analysis_checklist.csv", index=False)
print("META-ANALYSIS CHECKLIST")
print("=" * 80)
print(ma_checklist.to_string(index=False))
print("\n✓ Checklist saved to outputs/module_07/meta_analysis_checklist.csv")

---

## Congratulations!

You've completed **Module 07: Meta-Analysis Basics**. You can now:

✓ Determine when meta-analysis is appropriate  
✓ Calculate effect sizes (Cohen's d, OR, r)  
✓ Implement fixed-effect and random-effects models  
✓ Assess heterogeneity (I², Q, τ²)  
✓ Create and interpret forest plots  
✓ Detect publication bias (funnel plots, Egger's test)  
✓ Report meta-analysis results professionally  

**Next Module**: Research Communication & Writing  
**File**: `08_research_communication_writing.ipynb`

---